# STAT628 Module1 

# BodyFatCalculator Executive Summary

Author: Shuyi Qu, Guoli Liu, Linhai Zhang<br>

Note: In the R code part, many packages will be used, please make sure you have them installed.

## Introduction to analysis
Body fat percentage(BFP), which is the total mass of fat divided by total body mass, is an important measurement to describe people's healthy
condition. But it could be hard to calculate this measurement accurately. So the goal of this analysis is to come up with a simple and somehow
accurate "rule of thumb" of deteriming body fat percentage of males based on readily available clinical measurements.

## Background information about the data
The data set contains measurements from 252 men who had their body fat percentage accurately measured via underwater weighing.
It has 252 rows for 252 men and 17 columns for 1 index and 16 body measurements. "BODYFAT" indicates the accurate BFP of each person, while "DENSITY" is an alternative measurement to "BODYFAT". The total 16 body measurements contains such as weight, height and age.

## Model statement
Our goal is to find a rule of thumb for BFP, which means our model should be simple, easy to understand and remember. For this reason, we will choose the linear model. Because linear model is overall the simplest statistical model with the best interpretability. What's more, we will make the assumption that the random effect in the model has a normal distribution. Because BFP may be effected by many underlying aspects. Some of them will have positive effects, some will have negative effects. So Normal distribution can describe the add-up effect of these aspects best from its property.

## Data wrangling
In this part, we mainly focus on two things:
1. Whether there is any variable that distinguishs the data?
2. Whether there is any wrong-recorded data point or odd data point?

For the first question, by our common sense, we know that people's general healthy condition changes when aging, so we want to investigate whether there are any group effect between different age ranges. To investigate, we do a box-plot with different age ranges. (Here I do not put the box-plot in this summary, you can check in the images folder, it is p1.1.1.) From the box-plot we can observe that with different age ranges, variables like BODYFAT, HEIGHT, WEIGHT are basically of the same pattern. So we can believe that age does not have special effect.

For the second question, we would like to use a summary of every variable to see if there are any extreme records.<br>
Here we only show the summary of 4 variables that odd records occur

In [26]:
bodyfat = read.csv("data/BodyFat.csv")
summary(bodyfat[,c("BODYFAT","AGE","HEIGHT","WEIGHT")])

    BODYFAT           AGE            HEIGHT          WEIGHT     
 Min.   : 0.00   Min.   :22.00   Min.   :29.50   Min.   :118.5  
 1st Qu.:12.80   1st Qu.:35.75   1st Qu.:68.25   1st Qu.:159.0  
 Median :19.00   Median :43.00   Median :70.00   Median :176.5  
 Mean   :18.94   Mean   :44.88   Mean   :70.15   Mean   :178.9  
 3rd Qu.:24.60   3rd Qu.:54.00   3rd Qu.:72.25   3rd Qu.:197.0  
 Max.   :45.10   Max.   :81.00   Max.   :77.75   Max.   :363.1  

By the summary, we detect 5 odd points:
1. 42th  : HEIGHT is 29.5
2. 39ht  : WEIGHT is 363.15
3. 79th  : AGE is 81
4. 128th : BODYFAT is 0
5. 216th : BODYFAT is 45.1 (nearly impossible for human)

Then we make a quantile table of other main variables of these 5 data point to see if they are really odd:
<img src="images/p1.2.1.png">

From the quantile table, we can see that for the 39th and 79th points, although they have one extreme value variable(363.15 weight and 89 age), their other variables are within the normal range. So they are regular record. But for other 3 points, their value of other variables are not coordinate with their extreme variable value. So we delete them.

## Statistical analysis
In this part, we will omit the R code, because they would occupy lots of pages if we did not do it.<br>
But you can still check the full version R code in the repos.

The first thing we want to do is to use stepwise regression to delete variables to make our model simple. In this analysis we use BIC as the criterion. Because BIC tends to keep less variables than AIC.
Here is a final summary table of this part:
<img src="images/p2.2.1.png">

So we can see that only WEIGHT, ABDOMEN, WRIST and FOREARM these 4 predictors remain.<br>
Next we would like to try all combination of these 4 predictors<br>
and calculate all kinds of statistical criteria about them to find the best mode.<br>
Here is the final result:
<img src="images/p2.2.2.png">

We can see that there are two model perform that stand out: WEIGHT+ABDOMEN and ABDOMEN+WRIST.<br>
However, weight and abdomen have a strong collinearity. Consider BIC, adjusted r-square and quantile of residuals, we select abdomen and wrist as independent variables.<br>
So the model is:

In [28]:
bodyfat = bodyfat[-c(42,128,216),]
fit = lm(BODYFAT~ABDOMEN+WRIST,data = bodyfat)
fit
#quantile(fit$reisduals,probs = seq(0,1,0.1))


Call:
lm(formula = BODYFAT ~ ABDOMEN + WRIST, data = bodyfat)

Coefficients:
(Intercept)      ABDOMEN        WRIST  
    -8.1866       0.6888      -2.0128  


By anove we notice that while the intercept is not significant, the two predictors are very significant.<br>
And the residuals perform good with 80% of them within [-5,+5].

## Model interpretation
In order to make our model easy to calculate and remember, we only keep one significant digit,<br>
so our model is:<br>
$$ \text{(Body Fat %)}_i = -7 + 0.7 \cdot \text{Abdomen}_{i} - 2 \cdot \text{Wrist}_{i} + \epsilon_i \quad{}  $$

The coefficients of these two predictors are coordinate with our common sense, we can interpret them in this way.

The parameter of abdomen is 0.7, which means we control the wrist in a constant number, if abdomen increases 1-centimeter circumference, then body fat percent would increase by 0.7.

The parameter of wrist is tricky, -2. And the negative number does make sense. When we want to detect body fat percentage of a person, we should take the situation of skeleton into consideration and the circumference of wrist reflect the phenomenon. If wrist increases 1-centimeter and control the abdomen in a fixed number, which means this guy has a strong skeleton. So the body fat percentage would decrease by 2.


## Model diagnostics
In the model diagnostics, we care about the normality and Guass-Markov assumption of the error term most. So we will focus do test and make plots on the residuals.<br>
<img src="images/p3.1.1.png">

From the shapiro-wiki test we could see that the p-value is 0.1294, which meanse the assumption is not rejected, the test is passed.

In general, the model passes the test. The normality assumption and Guass-Markov assumption are held for this model, except the distribution of residuals are a little heavy-tailed.

## Strengths and weaknesses of analysis
### Strengths
There are overall 3 strengths about our analysis:<br>
1.The model is simple. Our model is a linear model under Gauss-Markov assumption with a normal error term. This is the most simple and also widely-used statistical model we could have. In our model, only two predictors are included, which makes our model easy to calculate and remember.<br>
2.The model is accurate for the majority of people. By the quantiles summary of model residuals, we can see that the 10% quantile is -4.67 and 90% quantile is 5.7. This means that when using this model, approximately the 80% of people will have their results within a [-5,+5] interval of the true value.<br>
3.The model has a good interpretability. By the model interpretation part we can see, the coefficients of our predictors are accord with our common sense.<br>

### Weaknesses
There are overall 3 strengths about our analysis:<br>
1.The intercept is not statistically significant. This means the intercept may change a lot when data changes.<br>
2.Model fails for a small group of people(20%). The 0% quantile and 100% quantile of residuals are -16.51 and 10.63, which means that for some extreme situations, the model will fail.<br>
3.The two predictors are not very robust when measuring and not easy to remember. We know that when measuring the wrist and abdomen of one person, errors may occur. This will affect accuracy of the model.<br>

## Conclusion
Overall, our analysis on this topic works in the following steps.<br>
1.Analyze the data with tables and plots, delete 3 odd points.<br>
2.Decide to use the linear model by the goal: obtain a "rule of thumb".<br>
3.Use stepwise regression with BIC as the criterion and get a 4-predictor model. Then try all combinations of those 4 predictors to get the best one.<br>
4.Do model diagnostics and summary the strengths and weaknesses of the model.<br>

Finally, our model is:<br>
$$ \text{(Body Fat %)}_i = -7 + 0.7 \cdot \text{Abdomen}_{i} - 2 \cdot \text{Wrist}_{i} + \epsilon_i, \quad{}  $$
where<br>
$$ \epsilon_i \sim N(0,\sigma^2) $$
It is a simple, quite accurate but not well robust model.

## Contributions
For this project, our group totally meet up for 4 times, 2 times for face-to-face meet and 2 times for online meet.<br>
Our project can be divided by 4 parts:<br>
1. The idea of how analysis goes on is decided by all 3 of us.
2. For the R code part, Shuyi Qu and Guoli Liu did the main job.
3. For the Jupyter notebook, Linhai Zhang did the main job.
4. For the presentation, Shuyi Qu did the main job.

<br>We think overall it is good team work experience.